In [1]:
from projects.NeuralForceField.train import *
import numpy as np
from sklearn.utils import shuffle
from projects.NeuralForceField.graphs import * 
import json

In [2]:
# ethanol data 
train = np.load('/home/wwj/data/ethanol_ccsd_t-train.npz')
xyz_data = np.dstack((np.array([train.f.z]*1000).reshape(1000, -1, 1), np.array(train.f.R)))
force_data = train.f.F
energy_data = train.f.E.squeeze() - train.f.E.mean()
smiles_data = ["COC"] * 1000

In [3]:
# initialize parameters
par = dict()

par["git_commit"] = "9129a434b318ab26ee890281f531f304fb499ca7"
par["model_type"] = "schnet"
par["n_filters"] = 256
par["n_gaussians"] = 32
par["n_atom_basis"] = 256
par["optim"] = 1e-4
par["scheduler"] = True
par["train_percentage"] = 0.8
par["T"] = 10
par["batch_size"] = 100
par["cutoff"] = 5.0 
par["max_epoch"] = 1000
par["trainable_gauss"] = True
par["rho"] = 0.1
par["eps"] = 1e-1

graph_data = load_graph_data(xyz_data=xyz_data, energy_data=energy_data, smiles_data=smiles_data,
                             batch_size=par["batch_size"], cutoff=par["cutoff"], 
                             force_data=force_data, au_flag=False)

Finished Batching 10 Batches


In [4]:
model = Model(par=par, device=0, job_name="test", train_flag=False, root='./log/')
model.load_model("./log/test_0314/model.pt")

# get species 
species = smiles_data[0]
xyz = np.array(xyz_data[0][:, 1:4])
r = np.array(xyz_data[0][:, 0])
traj = NVE(species=species, xyz=xyz, r=r, model=model.model, device=0, dir_loc="./log/test_0314/", steps=50)

RuntimeError: cuda runtime error (38) : no CUDA-capable device is detected at /opt/conda/conda-bld/pytorch_1535490206202/work/aten/src/THC/THCGeneral.cpp:74

In [ ]:
# smaple based on species 

def sample(model):

    species_dict, r_list, xyz_list = parse_species_geom(3, model.graph_data)

    for species in species_dict:
        index = np.random.choice(species_dict[species])

        xyz = xyz_list[index]
        r = r_list[index][:,0]

        NVE(species=species, xyz=xyz, r=r, device= model.device, dir_loc=model.dir_loc, model=model.model)
        
sample(model=model, steps = 100)